In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

#chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

#go to the site
url = 'https://squaredle.app/'
driver.get(url)

#wait 10 seconds before starting anything
wait = WebDriverWait(driver, 10)

#find body of the page 
body = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

#finish the tutorial

#play
body.send_keys('play')
body.send_keys(Keys.RETURN)

time.sleep(3)

#find
body.send_keys('find')
body.send_keys(Keys.RETURN)

time.sleep(3)

#tutorial
body.send_keys('tutorial')
body.send_keys(Keys.RETURN)

time.sleep(5)

#scrape the main game grid
#pase html content to extract the letters. get the first 16 outputs (main grid)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
letter_containers = soup.find_all('div', class_='letterContainer')
letters = [container.find('div', class_='unnecessaryWrapper').text.strip() for container in letter_containers][:16]

print(letters)



In [ ]:
#convert the 4x4 grid into a 2d array
letters = [letter.lower() for letter in letters]
group = [letters[:4], letters[4:8], letters[8:12], letters[12:]]
print(group)

In [ ]:
#get all the words from the corpus that are longer than 3 chars (squaredle allows only 4 letter words and up, max 16 letters)
with open('words.txt', 'r') as file:
    words = [line.strip() for line in file]

words = [word for word in words if len(word) > 3 and len(word) < 17]
print(len(words))

#filter to only have words with letters in the grid
words = [word for word in words if all(l in letters for l in word)]
len(words)

In [ ]:
#define a trie class to check if a string is a valid prefix of any word
#faster processing - prune unnecessary checks

class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_word = False
    
class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_word = True

    #check if there is any branch in the trie structure with a certain prefix
    def check_prefix(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return False
            node = node.children[char]
        return True

In [ ]:
#left right up down diagonal
directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]

def dfs(x, y, curr, visited, trie, candidates):
    #if words is in the corpus, add it as a potential candidate
    if trie.check_prefix(curr) and curr in words:
        candidates.add(curr)
        
    #traverse the grid in the possible directions
    for dx, dy in directions:
        nx, ny = x+dx, y+dy
        #if within grid bounds, continue traversing
        if 0 <= nx < len(group) and 0 <= ny < len(group[0]):
            if(nx, ny) not in visited:
                visited.add((nx, ny))
                dfs(nx, ny, curr+group[nx][ny], visited, trie, candidates)
                visited.remove((nx, ny))

#find all possible words using the letters in grid
def find_words():
    candidates = set()
    trie = Trie()
    #create a trie using all the possible words
    for word in words:
        trie.insert(word)
    #iterate through the paths through the grid and add matched words
    for i in range(len(group)):
        for j in range(len(group[0])):
            visited = set()
            visited.add((i, j))
            dfs(i, j, group[i][j], visited, trie, candidates)
    return candidates

potential_words = find_words()
print(len(potential_words))

In [ ]:
print(potential_words)

In [ ]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

#bonus word popups
def close_popups():
    ids = ["wordOfTheDay", "bonusWordDialog"]

    for id in ids:
        try:
            popup = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.ID, id))
            )
            close_btn = popup.find_element(By.CSS_SELECTOR, "a.closeBtn")

            #if close button is interactable, click it
            WebDriverWait(driver, 2).until(EC.element_to_be_clickable(close_btn))
            driver.execute_script("arguments[0].click();", close_btn)
            time.sleep(0.5)

        except (TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException):
            #if no popup shown or button isn't clickable, continue sending the rest of the words
            pass

for word in potential_words:
    #check if a popup showed up
    close_popups()
    #send each of the potential words to the site.
    body = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    body.send_keys(word)
    body.send_keys(Keys.RETURN)
    # print(f"Sent word: {word}")
    time.sleep(1)
